In [1]:
import threading
import time
import pandas as pd
import os

num_threads = 3

import os
from dotenv import load_dotenv
load_dotenv()
file_name = os.environ.get("file_name")
directory_path = 'C:\\programming_projects\\RAG_fine_tune\\RAG_pipeline_ASU_website\\data\\'
file_path = directory_path + 'chunked_' + file_name

In [2]:
import threading
import time
import pandas as pd
import os
from dotenv import load_dotenv

def worker(thread_id, all_data, start_index, num_rows=100):
    """Worker function that processes a chunk of the DataFrame and saves to CSV."""
    print(f"Thread {thread_id}: Starting at index {start_index}")
    end_index = start_index + num_rows
    thread_data = all_data.iloc[start_index:end_index]  # Use .iloc for positional indexing
    filename = f"thread_{thread_id}_data_{start_index}.csv"
    thread_data.to_csv(filename, index=False)
    print(f"Thread {thread_id}: Finished processing rows {start_index} to {end_index - 1} and saved to {filename}")

if __name__ == "__main__":
    threads = []
    num_threads = 3
    rows_per_thread = 100

    try:
        all_data = pd.read_csv(file_path, nrows=400)
        total_rows = len(all_data)
        start_indices = [i * rows_per_thread for i in range(num_threads)]

        for i in range(num_threads):
            print(i, "start:", (i * 100), "end:", ((i+1) * 100))
            start_index = (i * 100) ##start_indices[i]
            end_index = start_index + rows_per_thread
            if start_index < total_rows:
                # Ensure we don't go beyond the total number of rows
                actual_end_index = min(end_index, total_rows)
                thread_data = all_data.iloc[start_index:actual_end_index].copy() # Pass a slice of the DataFrame
                thread = threading.Thread(target=worker, args=((i + 1), thread_data, 0)) # Each thread gets its slice starting at index 0
                threads.append(thread)
                thread.start()
            else:
                print(f"Warning: Start index {start_index} is out of bounds for thread {i + 1}")

        # Wait for all threads to complete
        for thread in threads:
            thread.join()

        print("All threads have finished and saved their DataFrames.")

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

0 start: 0 end: 100
Thread 1: Starting at index 0
1 start: 100 end: 200
Thread 2: Starting at index 0
2 start: 200 end: 300
Thread 3: Starting at index 0
Thread 2: Finished processing rows 0 to 99 and saved to thread_2_data_0.csv
Thread 1: Finished processing rows 0 to 99 and saved to thread_1_data_0.csv
Thread 3: Finished processing rows 0 to 99 and saved to thread_3_data_0.csv
All threads have finished and saved their DataFrames.


In [5]:
import threading
import time
import pandas as pd
import os
from dotenv import load_dotenv

def worker(thread_id, all_data, row_counter, lock, rows_per_thread=100):
    """Worker function that continuously picks and processes chunks of the DataFrame."""
    while True:
        with lock:
            start_index = row_counter.value
            row_counter.value += rows_per_thread
            end_index = start_index + rows_per_thread

        if start_index >= len(all_data):
            print(f"Thread {thread_id}: No more rows to process.")
            break

        actual_end_index = min(end_index, len(all_data))
        thread_data = all_data.iloc[start_index:actual_end_index].copy()
        filename = f"thread_{thread_id}_data_{start_index}.csv"
        thread_data.to_csv(filename, index=False)
        print(f"Thread {thread_id}: Processed rows {start_index} to {actual_end_index - 1} and saved to {filename}")
        time.sleep(0.1) # Optional: Add a small delay to avoid tight looping

if __name__ == "__main__":
    threads = []
    num_threads = 3
    rows_per_thread = 100

    load_dotenv()
    file_name = os.environ.get("file_name")
    directory_path = 'C:\\programming_projects\\RAG_fine_tune\\RAG_pipeline_ASU_website\\data\\'
    file_path = directory_path + 'chunked_' + file_name
    print(file_path)

    try:
        all_data = pd.read_csv(file_path, nrows=400)
        total_rows = len(all_data)
        print(f"Total rows: {total_rows}, using {num_threads} threads.")

        # Shared counter to keep track of the next starting row
        row_counter = threading.Value('i', 0)
        lock = threading.Lock() # Lock to protect access to the counter

        # Create and start the threads
        for i in range(num_threads):
            thread = threading.Thread(target=worker, args=(i + 1, all_data, row_counter, lock, rows_per_thread))
            threads.append(thread)
            thread.start()

        # Wait for all threads to complete
        for thread in threads:
            thread.join()

        print("All threads have finished processing the data.")

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

C:\programming_projects\RAG_fine_tune\RAG_pipeline_ASU_website\data\chunked_cleaned_ASU_webpage_04_07_2025.csv
Total rows: 400, using 3 threads.
An error occurred: module 'threading' has no attribute 'Value'


In [6]:
import threading

try:
    counter = threading.Value('i', 0)
    print("threading.Value is available.")
except AttributeError as e:
    print(f"Error: {e}")

Error: module 'threading' has no attribute 'Value'


In [7]:
import sys
print(sys.version)

3.12.9 (tags/v3.12.9:fdb8142, Feb  4 2025, 15:27:58) [MSC v.1942 64 bit (AMD64)]


In [10]:
!pip install --upgrade threading

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement threading (from versions: none)
ERROR: No matching distribution found for threading


In [11]:
import datetime
import requests
import re
ASU_key = os.environ.get("ASU_key")
LLM_url = os.environ.get("LLM_url")

def cqa_api(chunked_df, i, ASU_key, LLM_url):
    print(datetime.datetime.now())
    df_out = pd.DataFrame([])
    cleaned_string = chunked_df['cleaned_text'].loc[i]

    chunk = chunked_df['chunked_text'].loc[i]
    title = chunked_df['title'].loc[i]
    url = chunked_df['url'].loc[i]
    chunked_word_count = chunked_df['chunked_word_count'].loc[i]
    orig_word_count = chunked_df['orig_word_count'].loc[i]
    ##########################################
    ## text type
    bearer_token = ASU_key
    json_payload = {
        "query": "what is the topic of the following text from: {cleaned_string}? only respond with the topic, no other text. Please make the topic 3 words or less".format(cleaned_string=cleaned_string),
        "model_provider": "gcp-deepmind",
        "model_name": "geminiflash2",
    }
    headers = {
        "Authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    try:
        response = requests.post(LLM_url, headers=headers, json=json_payload)
        response.raise_for_status()
        result_document_section = response.json().get("response")
        # print("result:", result_document_section)
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

    ##########################################
    ## questions

    query = """given that the following text from the webpage {title} on url {url}, here is a text chunk limited to 500 words:\n {chunk}\n\n 
                what are some good questions to ask about the text chunk? Please respond with a question and 3 different answers for each question.  There should be a total of 3 questions, with having 3 different answers (for a total of 9 unique answers).

                the questions need to be well defined. Try to use the text as much as possible when crafting the answer. Answers need to be at least 2 sentences long. Do not use the phrase "The text," and avoid similar language. Rephrase the question in the answer.

                Please use the following format for the response:

                **Question 1:**
                **Question 1 Answer 1:**
                **Question 1 Answer 2:**
                **Question 1 Answer 3:**

                **Question 2:**
                **Question 2 Answer 1:**
                **Question 2 Answer 2:**
                **Question 2 Answer 3:**

                **Question 3:**
                **Question 3 Answer 1:**
                **Question 3 Answer 2:**
                **Question 3 Answer 3:**
                """.format(
                    title = title,
                    url = url,
                    chunk=chunk)

    json_payload = {
        "query": query,
        "model_provider": "gcp-deepmind",
        "model_name": "geminiflash2",
    }
    headers = {
        "Authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    try:
        response = requests.post(LLM_url, headers=headers, json=json_payload)
        response.raise_for_status()
        result = response.json().get("response")
        # print("result:", result)
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    
    ###################################################
    ## save out
    parts = result.split("**")  # Split the string at **
    parts_no = [[2, 4], [2, 6], [2,8], [10, 12], [10, 14], [10, 16], [18, 20], [18, 22], [18, 24]]


    for jj in range(0, 9):
        try:
            pt1 = parts_no[jj][0]
            pt2 = parts_no[jj][1]

            Question = re.sub(r'[^a-zA-Z0-9.,!?\s]', ' ', str(parts[pt1]))
            Question = Question.replace(r'\s+', ' ').strip()
            Answer = re.sub(r'[^a-zA-Z0-9.,!?\s]', ' ', str(parts[pt2]))
            Answer = Answer.replace(r'\s+', ' ').strip()


            Q1 = pd.DataFrame(data={'section':[result_document_section],
                                    'title':[title],
                                    'url': [url],
                                    'document_type':['web page'],
                                    'chunked_word_count':[chunked_word_count],
                                    'orig_word_count':[orig_word_count],
                                    'contex': [chunk],
                                    'question':[Question],
                                    'answer':[Answer]
                                    })
            df_out = pd.concat([Q1, df_out], ignore_index=True)

        except Exception as e:
            print(f"Unexpected error: {e}")
    
    return(df_out)

In [12]:
all_data = pd.read_csv(file_path, nrows=400)

cqa_api(chunked_df = all_data, i = 399, ASU_key = ASU_key, LLM_url = LLM_url)

2025-04-11 10:40:41.042222


,section,title,url,document_type,chunked_word_count,orig_word_count,contex,question,answer
0,Broadway season announcement,ASU Gammage brings the best of Broadway to the...,https://news.asu.edu/20250225-arts-humanities-...,web page,500,989,ASU Gammage has announced its 2025 2026 Desert...,What makes Suffs a notable addition to the A...,Winning the Outer Critics Circle Award for Bes...
1,Broadway season announcement,ASU Gammage brings the best of Broadway to the...,https://news.asu.edu/20250225-arts-humanities-...,web page,500,989,ASU Gammage has announced its 2025 2026 Desert...,What makes Suffs a notable addition to the A...,"The musical is created by Shaina Taub, making ..."
2,Broadway season announcement,ASU Gammage brings the best of Broadway to the...,https://news.asu.edu/20250225-arts-humanities-...,web page,500,989,ASU Gammage has announced its 2025 2026 Desert...,What makes Suffs a notable addition to the A...,Suffs is a notable addition because it direct...
3,Broadway season announcement,ASU Gammage brings the best of Broadway to the...,https://news.asu.edu/20250225-arts-humanities-...,web page,500,989,ASU Gammage has announced its 2025 2026 Desert...,Which shows are part of the ASU Gammage 2025 2...,"The season also includes Clue, which runs fr..."
4,Broadway season announcement,ASU Gammage brings the best of Broadway to the...,https://news.asu.edu/20250225-arts-humanities-...,web page,500,989,ASU Gammage has announced its 2025 2026 Desert...,Which shows are part of the ASU Gammage 2025 2...,Another show featured in the season is Juli...
5,Broadway season announcement,ASU Gammage brings the best of Broadway to the...,https://news.asu.edu/20250225-arts-humanities-...,web page,500,989,ASU Gammage has announced its 2025 2026 Desert...,Which shows are part of the ASU Gammage 2025 2...,"The lineup includes Suffs, a Tony Award winn..."
6,Broadway season announcement,ASU Gammage brings the best of Broadway to the...,https://news.asu.edu/20250225-arts-humanities-...,web page,500,989,ASU Gammage has announced its 2025 2026 Desert...,When do new season subscriptions for the ASU G...,Current season ticket holders have an advantag...
7,Broadway season announcement,ASU Gammage brings the best of Broadway to the...,https://news.asu.edu/20250225-arts-humanities-...,web page,500,989,ASU Gammage has announced its 2025 2026 Desert...,When do new season subscriptions for the ASU G...,"Although subscriptions go on sale on April 7, ..."
8,Broadway season announcement,ASU Gammage brings the best of Broadway to the...,https://news.asu.edu/20250225-arts-humanities-...,web page,500,989,ASU Gammage has announced its 2025 2026 Desert...,When do new season subscriptions for the ASU G...,"New season subscriptions go on sale Monday, Ap..."


In [7]:
import threading
import time
import pandas as pd
import os
from dotenv import load_dotenv
from queue import Queue, Empty

def worker(thread_id, all_data, task_queue, output_queues):
    """Worker function that gets tasks from the queue and processes them."""
    while True:
        try:
            start_index, end_index = task_queue.get(timeout=1)
            thread_data = all_data.iloc[start_index:end_index].copy()
            output_queues[thread_id - 1].put((start_index, thread_data))  # Send start index with the data
            print(f"Thread {thread_id}: Processed rows {start_index} to {end_index - 1} and added to queue.")
            task_queue.task_done()
            time.sleep(0.1)
        except Empty:
            print(f"Thread {thread_id}: Task queue is empty, exiting.")
            break
        except Exception as e:
            print(f"Thread {thread_id}: An error occurred: {e}")
            break

def writer_worker(queue, thread_id):
    """Worker function that takes dataframes and their start index from a queue and writes them to CSV files."""
    while True:
        try:
            start_index, thread_data = queue.get(timeout=1)
            if thread_data.empty:  # Signal to stop
                print(f"Writer Thread {thread_id}: Received stop signal.")
                queue.task_done()
                break
            filename = f"thread_{thread_id}_data_{start_index}.csv"  # Use the received start_index
            thread_data.to_csv(filename, index=False)
            print(f"Writer Thread {thread_id}: Received and saved data to {filename}")
            queue.task_done()
        except Empty:
            print(f"Writer Thread {thread_id}: Writer queue empty, continuing to listen...")
        except Exception as e:
            print(f"Writer Thread {thread_id}: An error occurred: {e}")
            break

if __name__ == "__main__":
    threads = []
    writer_threads = []
    num_threads = 3
    rows_per_thread = 100

    load_dotenv()
    file_name = os.environ.get("file_name")
    directory_path = 'C:\\programming_projects\\RAG_fine_tune\\RAG_pipeline_ASU_website\\data\\'
    file_path = directory_path + 'chunked_' + file_name
    print(file_path)

    try:
        all_data = pd.read_csv(file_path, nrows=400)
        total_rows = len(all_data)
        print(f"Total rows: {total_rows}, using {num_threads} processing threads and {num_threads} writer threads.")

        task_queue = Queue()
        output_queues = [Queue() for _ in range(num_threads)]

        # Populate the task queue
        num_chunks = (total_rows + rows_per_thread - 1) // rows_per_thread
        for i in range(num_chunks):
            start = i * rows_per_thread
            end = min((i + 1) * rows_per_thread, total_rows)
            task_queue.put((start, end))

        # Create and start the processing threads
        for i in range(num_threads):
            thread = threading.Thread(target=worker, args=(i + 1, all_data, task_queue, output_queues))
            threads.append(thread)
            thread.daemon = True
            thread.start()

        # Create and start the writer threads
        for i in range(num_threads):
            writer_thread = threading.Thread(target=writer_worker, args=(output_queues[i], i + 1))
            writer_threads.append(writer_thread)
            writer_thread.daemon = True
            writer_thread.start()

        # Wait for all tasks to be processed
        task_queue.join()
        print("All tasks in the queue have been processed by worker threads.")

        # Signal writer threads to stop
        for q in output_queues:
            q.put((None, pd.DataFrame()))  # Send None for start_index as a stop signal

        # Wait for writer threads to finish
        for wt in writer_threads:
            wt.join()
        print("All writer threads have finished.")

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

C:\programming_projects\RAG_fine_tune\RAG_pipeline_ASU_website\data\chunked_cleaned_ASU_webpage_04_07_2025.csv
Total rows: 400, using 3 processing threads and 3 writer threads.
Thread 1: Processed rows 0 to 99 and added to queue.
Thread 2: Processed rows 100 to 199 and added to queue.
Thread 3: Processed rows 200 to 299 and added to queue.
Writer Thread 1: Received and saved data to thread_1_data_0.csv
Writer Thread 2: Received and saved data to thread_2_data_100.csv
Writer Thread 3: Received and saved data to thread_3_data_200.csv
Thread 1: Processed rows 300 to 399 and added to queue.
All tasks in the queue have been processed by worker threads.
Writer Thread 2: Received stop signal.
Writer Thread 3: Received stop signal.
Writer Thread 1: Received and saved data to thread_1_data_300.csv
Writer Thread 1: Received stop signal.
All writer threads have finished.


Thread 3: Task queue is empty, exiting.
Thread 2: Task queue is empty, exiting.
Thread 1: Task queue is empty, exiting.


In [ ]:
import threading
import time
import pandas as pd
import os
from dotenv import load_dotenv
from queue import Queue, Empty
import datetime
import requests
import re

ASU_key = os.environ.get("ASU_key")
LLM_url = os.environ.get("LLM_url")

def make_llm_request(query, api_key, api_url):
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    json_payload = {"query": query, "model_provider": "gcp-deepmind", "model_name": "geminiflash2"}
    try:
        response = requests.post(api_url, headers=headers, json=json_payload)
        response.raise_for_status()
        return response.json().get("response")
    except requests.exceptions.RequestException as e:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        print(f"{timestamp} API request error: {e}")
        return None
    except Exception as e:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        print(f"{timestamp} Unexpected error: {e}")
        return None

def parse_cqa_response(llm_response, title, url, chunked_word_count, orig_word_count, chunk):
    df_out = pd.DataFrame([])
    if llm_response:
        parts = llm_response.split("**")
        parts_no = [[2, 4], [2, 6], [2,8], [10, 12], [10, 14], [10, 16], [18, 20], [18, 22], [18, 24]]
        for jj in range(0, 9):
            try:
                pt1 = parts_no[jj][0]
                pt2 = parts_no[jj][1]
                Question = re.sub(r'[^a-zA-Z0-9.,!?\s]', ' ', str(parts[pt1])).replace(r'\s+', ' ').strip()
                Answer = re.sub(r'[^a-zA-Z0-9.,!?\s]', ' ', str(parts[pt2])).replace(r'\s+', ' ').strip()
                Q1 = pd.DataFrame(data={'title':[title], 'url': [url], 'document_type':['web page'], 'chunked_word_count':[chunked_word_count], 'orig_word_count':[orig_word_count], 'contex': [chunk], 'question':[Question], 'answer':[Answer]})
                df_out = pd.concat([Q1, df_out], ignore_index=True)
            except Exception as e:
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                print(f"{timestamp} Error parsing response: {e}")
    return df_out

def cqa_api(chunked_df, i, ASU_key, LLM_url):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    print(f"{timestamp} Processing row index: {i}")
    chunk = chunked_df['chunked_text'].iloc[i]
    title = chunked_df['title'].iloc[i]
    url = chunked_df['url'].iloc[i]
    chunked_word_count = chunked_df['chunked_word_count'].iloc[i]
    orig_word_count = chunked_df['orig_word_count'].iloc[i]

    qa_query = f"""given that the following text from the webpage {title} on url {url}, here is a text chunk limited to 500 words:\n {chunk}\n\n what are some good questions to ask about the text chunk? Please respond with a question and 3 different answers for each question. There should be a total of 3 questions, with having 3 different answers (for a total of 9 unique answers). ... (rest of the prompt)"""
    llm_response = make_llm_request(qa_query, ASU_key, LLM_url)

    return parse_cqa_response(llm_response, title, url, chunked_word_count, orig_word_count, chunk)

def worker(thread_id, all_data, task_queue, output_queues):
    """Worker function that gets tasks from the queue and processes them."""
    while True:
        try:
            start_index, end_index = task_queue.get(timeout=1)
            thread_data_chunk = all_data.iloc[start_index:end_index].copy()
            all_processed_dfs = []
            for i in thread_data_chunk.index: # Iterate through the rows of the chunk
                output_df = cqa_api(all_data, i, ASU_key, LLM_url)
                if not output_df.empty:
                    all_processed_dfs.append(output_df)

            if all_processed_dfs:
                combined_df = pd.concat(all_processed_dfs, ignore_index=True)
                output_queues[thread_id - 1].put((start_index, combined_df))
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                print(f"{timestamp} Thread {thread_id}: Processed rows {start_index} to {end_index - 1} and added {len(combined_df)} rows to queue.")
            else:
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                print(f"{timestamp} Thread {thread_id}: Processed rows {start_index} to {end_index - 1}, no output generated.")

            task_queue.task_done()
            time.sleep(0.1)
        except Empty:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            print(f"{timestamp} Thread {thread_id}: Task queue is empty, exiting.")
            break
        except Exception as e:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            print(f"{timestamp} Thread {thread_id}: An error occurred in worker: {e}")
            break

def writer_worker(queue, thread_id, all_data):
    """Worker function that takes dataframes and their start index from a queue and appends to a local dataframe until 10 original rows are processed, then writes to CSV."""
    local_df = pd.DataFrame()
    processed_original_rows = set()
    while True:
        try:
            start_index, thread_data = queue.get(timeout=1)
            if thread_data.empty:
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                print(f"{timestamp} Writer Thread {thread_id}: Received stop signal. Processed {len(processed_original_rows)} original rows.")
                break

            local_df = pd.concat([local_df, thread_data], ignore_index=True)
            for i in range(start_index, start_index + (len(thread_data) // 9)): # Assuming 9 output rows per input row
                processed_original_rows.add(i)

            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            print(f"{timestamp} Writer Thread {thread_id}: Received {len(thread_data)} rows, now processed {len(processed_original_rows)} original rows.")
            queue.task_done()

            if len(processed_original_rows) >= 10:
                output_start_index = min(processed_original_rows)
                filename = f"processed_data_thread_{thread_id}_start_{output_start_index}_10rows.csv"
                local_df.to_csv(filename, index=False)
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                print(f"{timestamp} Writer Thread {thread_id}: Saved {len(local_df)} rows (from 10 original) to {filename}.")
                local_df = pd.DataFrame()
                processed_original_rows = set()

        except Empty:
            pass
        except Exception as e:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            print(f"{timestamp} Writer Thread {thread_id}: An error occurred in writer: {e}")
            break

    # Save any remaining data if the stop signal is received before 10 rows
    if not local_df.empty:
        output_start_index = min(processed_original_rows) if processed_original_rows else "partial"
        filename = f"processed_data_thread_{thread_id}_start_{output_start_index}_partial.csv"
        local_df.to_csv(filename, index=False)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        print(f"{timestamp} Writer Thread {thread_id}: Saved remaining {len(local_df)} rows to {filename}.")

if __name__ == "__main__":
    threads = []
    writer_threads = []
    num_threads = 3
    rows_per_thread = 1  # Process one row at a time with the API call
    original_rows_per_writer_file = 10

    load_dotenv()
    file_name = os.environ.get("file_name")
    directory_path = 'C:\\programming_projects\\RAG_fine_tune\\RAG_pipeline_ASU_website\\data\\'
    file_path = directory_path + 'chunked_' + file_name
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    print(f"{timestamp} {file_path}")

    try:
        all_data = pd.read_csv(file_path, nrows=35) # Increased nrows for testing
        total_rows = len(all_data)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        print(f"{timestamp} Total rows: {total_rows}, using {num_threads} processing threads and {num_threads} writer threads.")

        task_queue = Queue()
        output_queues = [Queue() for _ in range(num_threads)]

        # Populate the task queue with individual row indices
        for i in range(total_rows):
            task_queue.put((i, i + 1)) # Each task is to process a single row

        # Create and start the processing threads
        for i in range(num_threads):
            thread = threading.Thread(target=worker, args=(i + 1, all_data, task_queue, output_queues))
            threads.append(thread)
            thread.daemon = True
            thread.start()

        # Create and start the writer threads
        for i in range(num_threads):
            writer_thread = threading.Thread(target=writer_worker, args=(output_queues[i], i + 1, all_data))
            writer_threads.append(writer_thread)
            writer_thread.daemon = True
            writer_thread.start()

        # Wait for all tasks to be processed
        task_queue.join()
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        print(f"{timestamp} All tasks in the queue have been processed by worker threads.")

        # Signal writer threads to stop
        for q in output_queues:
            q.put((None, pd.DataFrame()))

        # Wait for writer threads to finish
        for wt in writer_threads:
            wt.join()
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        print(f"{timestamp} All writer threads have finished.")

    except FileNotFoundError:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        print(f"{timestamp} Error: File not found at {file_path}")
    except Exception as e:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        print(f"{timestamp} An error occurred in main: {e}")

2025-04-11 11:40:36.643 C:\programming_projects\RAG_fine_tune\RAG_pipeline_ASU_website\data\chunked_cleaned_ASU_webpage_04_07_2025.csv
2025-04-11 11:40:36.646 Total rows: 35, using 3 processing threads and 3 writer threads.
2025-04-11 11:40:36.647 Processing row index: 0
2025-04-11 11:40:36.647 Processing row index: 1
2025-04-11 11:40:36.648 Processing row index: 2
2025-04-11 11:40:44.494 Thread 3: Processed rows 2 to 2 and added 9 rows to queue.
2025-04-11 11:40:44.495 Writer Thread 3: Received 9 rows, now processed 1 original rows.
2025-04-11 11:40:44.595 Processing row index: 3
2025-04-11 11:40:45.134 Thread 2: Processed rows 1 to 1 and added 9 rows to queue.
2025-04-11 11:40:45.135 Writer Thread 2: Received 9 rows, now processed 1 original rows.
2025-04-11 11:40:45.235 Processing row index: 4
2025-04-11 11:40:45.868 Thread 1: Processed rows 0 to 0 and added 9 rows to queue.
2025-04-11 11:40:45.868 Writer Thread 1: Received 9 rows, now processed 1 original rows.
2025-04-11 11:40:45.

2025-04-11 11:42:09.730 Thread 3: Task queue is empty, exiting.
2025-04-11 11:42:10.724 Thread 2: Task queue is empty, exiting.


In [19]:
def files_to_dataframe(directory):
    all_data = pd.DataFrame([])
    for file in os.listdir(directory):
        try:
            df_read = pd.read_csv(directory+file, encoding="utf_8_sig", low_memory=False)
            print(file, df_read.shape)
            df_read['filename'] = str(file)
            # df_read.drop_duplicates(inplace=True)
            ##print(file, df_read.shape)
            all_data = pd.concat([all_data, df_read])
        except Exception as e:
            print(f"Error reading {file}: {e}")
    return all_data

directory_path = 'C:\\programming_projects\\RAG_fine_tune\\RAG_pipeline_ASU_website\\'
df = files_to_dataframe(directory_path)
print( df.shape)
df.drop_duplicates(subset = ['answer'], inplace=True)
print('drop dups page_text:', df.shape)

Error reading 1_chunk_txt.ipynb: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2

Error reading 1_chunk_txt.py: Error tokenizing data. C error: Expected 1 fields in line 13, saw 3

Error reading 2_question_answer_context.ipynb: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2

Error reading 2_question_answer_context.py: Error tokenizing data. C error: Expected 1 fields in line 26, saw 2

Error reading 3_cqa_multi.ipynb: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2

Error reading data: [Errno 13] Permission denied: 'C:\\programming_projects\\RAG_fine_tune\\RAG_pipeline_ASU_website\\data'
processed_data_thread_1_start_0_10rows.csv (90, 8)
processed_data_thread_1_start_32_partial.csv (9, 8)
processed_data_thread_2_start_1_10rows.csv (90, 8)
processed_data_thread_2_start_31_partial.csv (18, 8)
processed_data_thread_3_start_2_10rows.csv (90, 8)
processed_data_thread_3_start_30_partial.csv (18, 8)
(315, 9)
drop dups page_text: (315, 